<a href="https://colab.research.google.com/github/Bahae9/Kaggle_competitions/blob/main/Loan_Approval_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c playground-series-s4e10
!unzip playground-series-s4e10.zip

  0% 0.00/1.45M [00:00<?, ?B/s]
100% 1.45M/1.45M [00:00<00:00, 28.0MB/s]
Archive:  playground-series-s4e10.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
!pip install optuna
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 14.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
import optuna
from sklearn.model_selection import cross_val_score

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
df = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
df

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58640,58640,34,120000,MORTGAGE,5.0,EDUCATION,D,25000,15.95,0.21,Y,10,0
58641,58641,28,28800,RENT,0.0,MEDICAL,C,10000,12.73,0.35,N,8,1
58642,58642,23,44000,RENT,7.0,EDUCATION,D,6800,16.00,0.15,N,2,1
58643,58643,22,30000,RENT,2.0,EDUCATION,A,5000,8.90,0.17,N,3,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58645 entries, 0 to 58644
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          58645 non-null  int64  
 1   person_age                  58645 non-null  int64  
 2   person_income               58645 non-null  int64  
 3   person_home_ownership       58645 non-null  object 
 4   person_emp_length           58645 non-null  float64
 5   loan_intent                 58645 non-null  object 
 6   loan_grade                  58645 non-null  object 
 7   loan_amnt                   58645 non-null  int64  
 8   loan_int_rate               58645 non-null  float64
 9   loan_percent_income         58645 non-null  float64
 10  cb_person_default_on_file   58645 non-null  object 
 11  cb_person_cred_hist_length  58645 non-null  int64  
 12  loan_status                 58645 non-null  int64  
dtypes: float64(3), int64(6), object

In [ ]:
def encode(df):
  df = df.replace({'A': 5, 'B': 4, 'C': 3, 'D': 2, 'E': 1, 'F': 0, 'G': -1})
  df = df.replace({'N': 0, 'Y': 1})
  df = pd.get_dummies(df, columns=['person_home_ownership', 'loan_intent'], prefix=['home_ownership', 'loan_intent'])
  return df

In [ ]:
df = encode(df)

<ipython-input-6-337341cb87c0>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({'A': 5, 'B': 4, 'C': 3, 'D': 2, 'E': 1, 'F': 0, 'G': -1})
<ipython-input-6-337341cb87c0>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({'N': 0, 'Y': 1})


In [ ]:
test = encode(test)
X_test_pred = test.copy()

<ipython-input-6-337341cb87c0>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({'A': 5, 'B': 4, 'C': 3, 'D': 2, 'E': 1, 'F': 0, 'G': -1})
<ipython-input-6-337341cb87c0>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({'N': 0, 'Y': 1})


In [ ]:
X = df.drop(columns=['loan_status'])
y = df['loan_status']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def objective_xgb(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
    }
    model = XGBClassifier(**params, random_state=42, eval_metric='auc')
    score = cross_val_score(model, X, y, n_jobs=-1, cv=5, scoring='roc_auc').mean()
    return score

def objective_lgbm(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
    }
    model = LGBMClassifier(**params, random_state=42)
    score = cross_val_score(model, X, y, n_jobs=-1, cv=5, scoring='roc_auc').mean()
    return score

def objective_rf(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
    }
    model = RandomForestClassifier(**params, random_state=42)
    score = cross_val_score(model, X, y, n_jobs=-1, cv=5, scoring='roc_auc').mean()
    return score

def objective_catboost(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'random_strength': trial.suggest_float('random_strength', 1, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 10),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'od_wait': trial.suggest_int('od_wait', 10, 50),
    }
    model = CatBoostClassifier(**params, random_state=42, eval_metric='AUC', verbose=False)
    score = cross_val_score(model, X, y, n_jobs=-1, cv=5, scoring='roc_auc').mean()
    return score

# Optimize CatBoost hyperparameters
study_catboost = optuna.create_study(direction='maximize')
study_catboost.optimize(objective_catboost, n_trials=30)  # Adjust n_trials as needed
print("Best CatBoost hyperparameters:", study_catboost.best_params)
# Optimize XGBoost hyperparameters
study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=30)  # Adjust n_trials as needed
print("Best XGBoost hyperparameters:", study_xgb.best_params)

# Optimize LightGBM hyperparameters
study_lgbm = optuna.create_study(direction='maximize')
study_lgbm.optimize(objective_lgbm, n_trials=30)  # Adjust n_trials as needed
print("Best LightGBM hyperparameters:", study_lgbm.best_params)

# Optimize Random Forest hyperparameters
study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(objective_rf, n_trials=30)  # Adjust n_trials as needed
print("Best Random Forest hyperparameters:", study_rf.best_params)


[I 2024-10-01 23:14:11,826] A new study created in memory with name: no-name-6ada3c83-bbb8-4290-bcc2-9e50053fe6c7
[I 2024-10-01 23:14:50,010] Trial 0 finished with value: 0.953314319764884 and parameters: {'iterations': 389, 'learning_rate': 0.1356181656089208, 'depth': 5, 'l2_leaf_reg': 9.037946029076725, 'border_count': 155, 'random_strength': 3.1441582494546205, 'bagging_temperature': 3.3885699691748004, 'od_type': 'IncToDec', 'od_wait': 15}. Best is trial 0 with value: 0.953314319764884.
[I 2024-10-01 23:15:41,362] Trial 1 finished with value: 0.9432246631104031 and parameters: {'iterations': 488, 'learning_rate': 0.270590898613817, 'depth': 7, 'l2_leaf_reg': 6.572606323702583, 'border_count': 177, 'random_strength': 9.004800975670854, 'bagging_temperature': 5.913635819089932, 'od_type': 'IncToDec', 'od_wait': 16}. Best is trial 0 with value: 0.953314319764884.
[I 2024-10-01 23:16:01,469] Trial 2 finished with value: 0.9471405295582411 and parameters: {'iterations': 178, 'learning_

Best CatBoost hyperparameters: {'iterations': 389, 'learning_rate': 0.1356181656089208, 'depth': 5, 'l2_leaf_reg': 9.037946029076725, 'border_count': 155, 'random_strength': 3.1441582494546205, 'bagging_temperature': 3.3885699691748004, 'od_type': 'IncToDec', 'od_wait': 15}


[I 2024-10-01 23:36:32,438] Trial 0 finished with value: 0.9467462569641508 and parameters: {'max_depth': 7, 'learning_rate': 0.2233636609900431, 'n_estimators': 163, 'min_child_weight': 9, 'subsample': 0.9886521242272591, 'colsample_bytree': 0.7695549480082858, 'gamma': 7.4425963625186, 'reg_alpha': 4.21790949759544, 'reg_lambda': 8.187082240920594}. Best is trial 0 with value: 0.9467462569641508.
[I 2024-10-01 23:36:35,394] Trial 1 finished with value: 0.9422617455217808 and parameters: {'max_depth': 4, 'learning_rate': 0.19205167659306524, 'n_estimators': 214, 'min_child_weight': 5, 'subsample': 0.8451638664821763, 'colsample_bytree': 0.8481439559665134, 'gamma': 9.587354631424947, 'reg_alpha': 5.5024788918802106, 'reg_lambda': 4.0642742409135435}. Best is trial 0 with value: 0.9467462569641508.
[I 2024-10-01 23:36:41,292] Trial 2 finished with value: 0.9402864774477291 and parameters: {'max_depth': 8, 'learning_rate': 0.06230169584657044, 'n_estimators': 382, 'min_child_weight': 9,

Best XGBoost hyperparameters: {'max_depth': 9, 'learning_rate': 0.08162442747596123, 'n_estimators': 408, 'min_child_weight': 6, 'subsample': 0.9606721160739966, 'colsample_bytree': 0.5246249337091624, 'gamma': 0.9778592845089441, 'reg_alpha': 1.3046095736816625, 'reg_lambda': 9.663682545922917}


[I 2024-10-01 23:39:50,698] Trial 0 finished with value: 0.9554862240922273 and parameters: {'num_leaves': 67, 'learning_rate': 0.14187443478386536, 'n_estimators': 296, 'max_depth': 5, 'min_child_samples': 25, 'subsample': 0.806388084282289, 'colsample_bytree': 0.5066068328462827, 'reg_alpha': 6.420269186541532, 'reg_lambda': 0.9814227636065775}. Best is trial 0 with value: 0.9554862240922273.
[I 2024-10-01 23:40:01,804] Trial 1 finished with value: 0.9543695787667135 and parameters: {'num_leaves': 207, 'learning_rate': 0.13669657576426977, 'n_estimators': 276, 'max_depth': 9, 'min_child_samples': 86, 'subsample': 0.8410422381297358, 'colsample_bytree': 0.7649909815741731, 'reg_alpha': 5.369676654110199, 'reg_lambda': 1.9105998930350143}. Best is trial 0 with value: 0.9554862240922273.
[I 2024-10-01 23:40:19,335] Trial 2 finished with value: 0.952951276093801 and parameters: {'num_leaves': 200, 'learning_rate': 0.049463178538089514, 'n_estimators': 411, 'max_depth': 9, 'min_child_samp

Best LightGBM hyperparameters: {'num_leaves': 225, 'learning_rate': 0.08245968867808884, 'n_estimators': 407, 'max_depth': 6, 'min_child_samples': 46, 'subsample': 0.5469000974065992, 'colsample_bytree': 0.6113458989028802, 'reg_alpha': 7.697039904785172, 'reg_lambda': 5.488324394636368}


[I 2024-10-01 23:45:30,592] Trial 0 finished with value: 0.9337541558695911 and parameters: {'n_estimators': 375, 'max_depth': 10, 'min_samples_split': 18, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.9337541558695911.
[I 2024-10-01 23:46:00,030] Trial 1 finished with value: 0.9153640526879435 and parameters: {'n_estimators': 132, 'max_depth': 10, 'min_samples_split': 20, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.9337541558695911.
[I 2024-10-01 23:46:57,310] Trial 2 finished with value: 0.9348539187655109 and parameters: {'n_estimators': 269, 'max_depth': 10, 'min_samples_split': 20, 'min_samples_leaf': 10}. Best is trial 2 with value: 0.9348539187655109.
[I 2024-10-01 23:47:08,647] Trial 3 finished with value: 0.906370224061272 and parameters: {'n_estimators': 121, 'max_depth': 3, 'min_samples_split': 12, 'min_samples_leaf': 2}. Best is trial 2 with value: 0.9348539187655109.
[I 2024-10-01 23:47:50,825] Trial 4 finished with value: 0.9282340657188456 and parameter

Best Random Forest hyperparameters: {'n_estimators': 269, 'max_depth': 10, 'min_samples_split': 20, 'min_samples_leaf': 10}


In [ ]:
# Best CatBoost hyperparameters: {'iterations': 389, 'learning_rate': 0.1356181656089208, 'depth': 5, 'l2_leaf_reg': 9.037946029076725, 'border_count': 155, 'random_strength': 3.1441582494546205, 'bagging_temperature': 3.3885699691748004, 'od_type': 'IncToDec', 'od_wait': 15}
# Best XGBoost hyperparameters: {'max_depth': 9, 'learning_rate': 0.08162442747596123, 'n_estimators': 408, 'min_child_weight': 6, 'subsample': 0.9606721160739966, 'colsample_bytree': 0.5246249337091624, 'gamma': 0.9778592845089441, 'reg_alpha': 1.3046095736816625, 'reg_lambda': 9.663682545922917}
# Best LightGBM hyperparameters: {'num_leaves': 225, 'learning_rate': 0.08245968867808884, 'n_estimators': 407, 'max_depth': 6, 'min_child_samples': 46, 'subsample': 0.5469000974065992, 'colsample_bytree': 0.6113458989028802, 'reg_alpha': 7.697039904785172, 'reg_lambda': 5.488324394636368}
# Best Random Forest hyperparameters: {'n_estimators': 269, 'max_depth': 10, 'min_samples_split': 20, 'min_samples_leaf': 10}

In [ ]:
cb_params = {'iterations': 389, 'learning_rate': 0.1356181656089208, 'depth': 5, 'l2_leaf_reg': 9.037946029076725, 'border_count': 155, 'random_strength': 3.1441582494546205, 'bagging_temperature': 3.3885699691748004, 'od_wait': 15}
xgb_params = {'max_depth': 9, 'learning_rate': 0.08162442747596123, 'n_estimators': 408, 'min_child_weight': 6, 'subsample': 0.9606721160739966, 'colsample_bytree': 0.5246249337091624, 'gamma': 0.9778592845089441, 'reg_alpha': 1.3046095736816625, 'reg_lambda': 9.663682545922917}
lgbm_params = {'num_leaves': 225, 'learning_rate': 0.08245968867808884, 'n_estimators': 407, 'max_depth': 6, 'min_child_samples': 46, 'subsample': 0.5469000974065992, 'colsample_bytree': 0.6113458989028802, 'reg_alpha': 7.697039904785172, 'reg_lambda': 5.488324394636368}
rf_params = {'n_estimators': 269, 'max_depth': 10, 'min_samples_split': 20, 'min_samples_leaf': 10}

In [ ]:
model1 = XGBClassifier(**xgb_params)
model2 = RandomForestClassifier(**rf_params)
model3 = LGBMClassifier(**lgbm_params)
model4 = CatBoostClassifier(**cb_params)
# Create a voting classifier
model = VotingClassifier(estimators=[('xgb', model1), ('rf', model2), ('lgbm', model3), ('cb', model4)], voting='soft')

# Fit the voting classifier
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict_proba(X_test)[:, 1]

# Evaluate the model
roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC AUC Score: {roc_auc}")

[LightGBM] [Info] Number of positive: 6708, number of negative: 40208
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003880 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1105
[LightGBM] [Info] Number of data points in the train set: 46916, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.142979 -> initscore=-1.790765
[LightGBM] [Info] Start training from score -1.790765
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

In [ ]:
model1 = XGBClassifier(**xgb_params)
model2 = RandomForestClassifier(**rf_params)
model3 = LGBMClassifier(**lgbm_params)
model4 = CatBoostClassifier(**cb_params)

model = VotingClassifier(estimators=[('xgb', model1), ('rf', model2), ('lgbm', model3), ('cb', model4)], voting='soft')

model.fit(X, y)

[LightGBM] [Info] Number of positive: 8350, number of negative: 50295
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002064 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1108
[LightGBM] [Info] Number of data points in the train set: 58645, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.142382 -> initscore=-1.795644
[LightGBM] [Info] Start training from score -1.795644
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.5246249337091624,
                                            device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None,
                                            gamma=0.9778592845089441,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_...
                                                     min_samples_split=20,
                                                     n_estimators=269)),
                             ('lgbm',
                              LGBMClassifier(colsample_bytree=0.6113458989028802,
                                             learning_rate=0.08245968867808884,
                                             max_depth=6, min_child_samples=46,
                                             n_estimators=407, num_leaves=225,
                                             reg_alpha=7.697039904785172,
                                             reg_lambda=5.488324394636368,
                                             subsample=0.5469000974065992)),
                             ('cb',
                              <catboost.core.CatBoostClassifier object at 0x7dcc43731510>)],
                 voting='soft')

In [ ]:
y_pred = model.predict(X_test_pred)

submission_df = pd.DataFrame({'id': test['id'], 'loan_status': y_pred})

submission_df['loan_status'] = submission_df['loan_status'].astype(int)

submission_df.to_csv('submission.csv', index=False)

In [ ]:
# !kaggle competitions submit -c playground-series-s4e10 -f submission.csv -m "Message"

## The best model is RandomForest alone with a public score: 0.87433